

```
# Ejemplo Introductorio
```
# Ejemplo Introductorio
No es necesario comprender el código. Sólo vamos a mostrar un ejemplo de lo que Ud. al final de este curso debiera hacer por cuenta propia.

### El estudio
Supongamos que trabaja para una compañía de vinos nacionales, y tiene información de psicoquímica (inputs) y sensorial (salida) de varios vinos y productos, algunos de los cuales son de la compañía y otros de la competencia.
La dataset consta de 4898 muestras de vinos y 12 atributos.

### La tarea
La tarea es diseñar una máquina que permita clasificar automáticamente un producto en la categoría de buena o mala calidad.
Dentro de los atributos, tenemos una evaluación sensorial en una escala de 1 a 10. 1 es malo, 10 es excelente.

### Procedimiento
En general, el procedimiento es el siguiente:
1. Importar los datos
2. Verificar datos perdidos o anómalos
3. Estudiar los datos (ver relaciones entre variables, crear nuevas variables predictoras, etc.)
4. Generar un set de datos para entrenar la máquina y otro para testear.
5. Diseñar la NN (arquitectura)
6. Entrenar el modelo (máquina) y evaluar su desempeño.
7. Evaluar el modelo con el set de test.
8. Volver a 5 paso 5 si es necesario.
--Fin.



Fuente de los datos:
https://archive.ics.uci.edu/dataset/186/wine+quality


In [56]:
import numpy as np
import pandas as pd

# vamos a importar los datos directamente desde github:
url = 'https://raw.githubusercontent.com/MAValle/deepLearning_intro_course/main/winequality_red.csv'
df = pd.read_csv(url)
df.head()

# Note que la primera columna es string. La vamos a aquitar puesto que aqui
# no nos interesa.

df = df.drop(['type'], axis=1)

df = df.dropna() # eliminamos instancias que tengan NA
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6463 entries, 0 to 6496
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed_acidity         6463 non-null   float64
 1   volatile_acidity      6463 non-null   float64
 2   citric_acid           6463 non-null   float64
 3   residual_sugar        6463 non-null   float64
 4   chlorides             6463 non-null   float64
 5   free_sulfur_dioxide   6463 non-null   float64
 6   total_sulfur_dioxide  6463 non-null   float64
 7   density               6463 non-null   float64
 8   pH                    6463 non-null   float64
 9   sulphates             6463 non-null   float64
 10  alcohol               6463 non-null   float64
 11  quality               6463 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 656.4 KB


In [57]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

# Importamos en este ejemplo, tensorflow:
# def:
# "TensorFlow is an end-to-end open source platform for machine learning.
# TensorFlow is a rich system for managing all aspects of a machine learning
# system; however, this class focuses on using a particular TensorFlow API to
# develop and train machine learning models. See the TensorFlow documentation
# for complete details on the broader TensorFlow system".
# Fuente: https://developers.google.com/machine-learning/crash-course/first-steps-with-tensorflow/toolkit#:~:text=TensorFlow%20is%20an%20end%2Dto,and%20train%20machine%20learning%20models.

# Vamos a seleccionar 80% de los datos para entrenar, y el resteante para testear
train_df = df.sample(frac=0.80, random_state=4)
test_df = df.drop(train_df.index)

In [58]:
# Para evitar problemas de distintas ESCALAS posibles en la bbdd,
# vamos a escalar todas las variables predictoras entre 0 y 1 en el set de train.

max_val = train_df.max(axis=0) # max value para cada columna.
min_val = train_df.min(axis=0)

rango = max_val - min_val
train_df = (train_df - min_val)/rango
test_df =  (test_df- min_val)/rango

In [59]:
# Ahora vamos a separar variables predictoras del target para train y test:

X_train = train_df.drop('quality',axis=1) # quitamos el target
X_test = test_df.drop('quality',axis=1) # definimos el target
y_train = train_df['quality']
y_test = test_df['quality']


# vamos a rescatar el numero de predictores que tenemos. Despues lo necesitamos
input_shape = [X_train.shape[1]]
print(input_shape)
print(np.any(np.isnan(X_test)))
print(np.any(np.isnan(y_test)))

[11]
False
False


## Creamos un modelo de NN
Aquí lo importante es comenzar con un modelo simple. La NN puede ser bastante demandante en términos de GPU y no queremos un modelo
sobredimensionado para un problema.

La arquitectura de la red la iremos viendo más adelante. Por lo pronto, sólo observe la "lógica" de operación para crear el modelo.

In [60]:
# Arquitectura:
# Vamos a crear una NN con 3 capas.
# La primera es la capa visible por donde ingresa la info del mundo experior (predictores). Tiene 11 neuronas.
# La tercera es la capa de salida que nos intentará predecir la calidad del vino. Tiene 1 neurona
# La segunda capa es la capa escondida que tendrá 8 neuronas.

mi_primer_modelo = Sequential()
mi_primer_modelo.add(Dense( units=64, activation='relu', input_shape=input_shape) )
mi_primer_modelo.add(Dense(units=1) )
mi_primer_modelo.summary()

# Como vemos, el summary nos da un resumen de la arquitectura.
# Num parametros de capa input a capa hidden: 11*64 = 704 + 64 bias = 768
# Num parametros capa hidden a capa output: 64 + 1 bias = 65

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 64)                768       
                                                                 
 dense_11 (Dense)            (None, 1)                 65        
                                                                 
Total params: 833
Trainable params: 833
Non-trainable params: 0
_________________________________________________________________


In [61]:
# Una vez creado el modelo, debemos compilarlo:
mi_primer_modelo.compile(loss='mean_absolute_error', optimizer='adam' )

In [63]:
# Y ahora esmtamos listos para entrenar el modelo:
historia = mi_primer_modelo.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=256, epochs=25)

Epoch 1/25
21/21 [==============================] - 0s 6ms/step - loss: 0.0991 - val_loss: 0.0955
Epoch 2/25
21/21 [==============================] - 0s 4ms/step - loss: 0.0981 - val_loss: 0.0949
Epoch 3/25
21/21 [==============================] - 0s 3ms/step - loss: 0.0973 - val_loss: 0.0949
Epoch 4/25
21/21 [==============================] - 0s 4ms/step - loss: 0.0968 - val_loss: 0.0948
Epoch 5/25
21/21 [==============================] - 0s 4ms/step - loss: 0.0963 - val_loss: 0.0945
Epoch 6/25
21/21 [==============================] - 0s 3ms/step - loss: 0.0960 - val_loss: 0.0943
Epoch 7/25
21/21 [==============================] - 0s 4ms/step - loss: 0.0956 - val_loss: 0.0943
Epoch 8/25
21/21 [==============================] - 0s 3ms/step - loss: 0.0954 - val_loss: 0.0940
Epoch 9/25
21/21 [==============================] - 0s 3ms/step - loss: 0.0952 - val_loss: 0.0945
Epoch 10/25
21/21 [==============================] - 0s 3ms/step - loss: 0.0949 - val_loss: 0.0939
Epoch 11/25
21/21 [

### Predicciones
Lo interesante después de entrenado el modelo, es que podemos utilizarlo una vez que nos llegue un nuevo producto.


In [66]:
# Hagamos que nos llegan tres nuevos producto con los siguiente atributos
nuevos_productos = X_test.iloc[0:3, :]
nuevos_productos

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol
0,0.264463,0.126667,0.216867,0.308282,0.059900,0.152778,0.377880,0.267785,0.217054,0.129213,0.061538
6,0.198347,0.160000,0.096386,0.098160,0.059900,0.100694,0.299539,0.150183,0.356589,0.140449,0.184615
9,0.355372,0.093333,0.259036,0.013804,0.058236,0.093750,0.283410,0.128976,0.387597,0.129213,0.400000


In [78]:
from tensorflow.python.framework.ops import prepend_name_scope
# Hacemos la prediccion:
preds = mi_primer_modelo.predict(nuevos_productos )

preds = preds*rango[11] + min_val[11]

print("Las predicciones de calidad para los productos serían ", preds)


1/1 [==============================] - 0s 25ms/step
Las predicciones de calidad para los productos serían  [[5.7638683]
 [5.2483687]
 [5.8770905]]
